# Dataloader

## Includes

In [ ]:
# mass includes
import os
import cv2
import pickle
import math
import numpy as np
import rawpy as rp
import torch as t
from torch.utils import data
from shutil import rmtree
from ipynb.fs.full.util import unifyBayerPtn, batchCrop, randHorFlip, randVerFlip
from ipynb.fs.full.ISPColor import *

## Paired set

In [ ]:
class PairedSet(data.Dataset):
    def __init__(self, opt):
        self.data_root = opt.data_root
        self.crop_size = opt.crop_size

        # find all paired samples
        self.file_list = [
            file for file in os.listdir(os.path.join(self.data_root, 'paired'))
            if '.pkl' in file
        ]
        self.file_list.sort()

    def __getitem__(self, index):
        # load a new sample
        file_path = os.path.join(self.data_root, 'paired',
                                 self.file_list[index])
        with open(file_path, 'rb') as pkl:
            sample = pickle.load(pkl)
        in_img = sample['zeroed']
        out_img = sample['expert']

        # random operations
        in_img, out_img = batchCrop([in_img, out_img], self.crop_size)
        in_img, out_img = randHorFlip([in_img, out_img])
        in_img, out_img = randVerFlip([in_img, out_img])

        # normalize
        in_img = np.float32(in_img) / 65535.0
        out_img = np.float32(out_img) / 65535.0

        # convert to tensor
        in_img = t.tensor(in_img).permute(2, 0, 1)
        out_img = t.tensor(out_img).permute(2, 0, 1)

        # convert to linear sRGB
        in_img = t.where(in_img <= 0.04045, in_img / 12.92,
                         ((in_img + 0.055) / 1.055)**2.4)

        return in_img, out_img

    def __len__(self):

        return len(self.file_list)

## Unpaired set

In [ ]:
class UnpairedSet(data.Dataset):
    def __init__(self, opt):
        self.data_root = opt.data_root
        self.crop_size = opt.crop_size

        # find all training samples
        self.file_list = []
        self.weight_list = []
        with open(os.path.join(self.data_root, 'unpaired', 'prob.txt'),
                  'r') as txt:
            lines = txt.readlines()
        for line in lines:
            file, _, weight, _ = line.split(' ')
            self.file_list.append(file + '.pkl')
            self.weight_list.append(float(weight))

    def __getitem__(self, index):
        # load a new sample
        file_path = os.path.join(self.data_root, 'unpaired',
                                 self.file_list[index])
        with open(file_path, 'rb') as pkl:
            sample = pickle.load(pkl)
        lin_img = sample['lin_img']

        # random operations
        lin_img = batchCrop(lin_img, self.crop_size)
        lin_img = randHorFlip(lin_img)
        lin_img = randVerFlip(lin_img)

        # convert to tensor
        lin_img = t.tensor(lin_img).permute(2, 0, 1)

        # generate non-linear sRGB
        nlin_img = t.where(lin_img <= 0.0031308, 12.92 * lin_img,
                           1.055 * (lin_img**(1 / 2.4)) - 0.055)

        return lin_img, nlin_img

    def __len__(self):

        return len(self.file_list)

## Denoising set

In [ ]:
class DenoiseSet(data.Dataset):
    def __init__(self, opt, cam_model, mode):
        self.data_path = os.path.join(opt.data_root, 'denoise', cam_model,
                                      mode)
        self.crop_size = opt.crop_size
        self.mode = mode

        self.file_list = [
            file for file in os.listdir(self.data_path) if '.pkl' in file
        ]
        self.file_list.sort()

    def __getitem__(self, index):
        # load a new sample
        file_path = os.path.join(self.data_path, self.file_list[index])
        with open(file_path, 'rb') as pkl:
            sample = pickle.load(pkl)
        noisy_raw = sample['noisy_raw']
        clean_raw = sample['clean_raw']
        noise_map = sample['variance']
        ilm_coes = sample['ilm_coes']
        cam2xyz = sample['cam2xyz']

        # random cropping
        if self.mode == 'train':
            noisy_raw, clean_raw, noise_map, ilm_coes = batchCrop(
                [noisy_raw, clean_raw, noise_map, ilm_coes], self.crop_size)
        else:
            noisy_raw, clean_raw, noise_map, ilm_coes = batchCrop(
                [noisy_raw, clean_raw, noise_map, ilm_coes],
                self.crop_size,
                centred=True)

        # normalize and convert to tensor
        noisy_raw = t.tensor(noisy_raw).permute(2, 0, 1)
        clean_raw = t.tensor(clean_raw).permute(2, 0, 1)
        noise_map = t.tensor(noise_map).permute(2, 0, 1)
        ilm_coes = t.tensor(ilm_coes).permute(2, 0, 1)
        cam2xyz = t.tensor(cam2xyz)

        return noisy_raw, clean_raw, noise_map, ilm_coes, cam2xyz

    def __len__(self):

        return len(self.file_list)

## Validation set

In [ ]:
class ValSet(data.Dataset):
    def __init__(self, opt, folder):
        self.data_path = os.path.join(opt.data_root, folder)

        # find all training samples
        self.file_list = [
            file for file in os.listdir(os.path.join(self.data_path))
            if '.dng' in file or '.DNG' in file
        ]
        self.file_list.sort()

    def __getitem__(self, index):
        # load a new sample
        file = self.file_list[index]
        file_path = os.path.join(self.data_path, file)
        with rp.imread(file_path) as raw_obj:
            cfa_data = raw_obj.raw_image_visible.copy()
            cfa_mask = raw_obj.raw_colors_visible
            blk_level = raw_obj.black_level_per_channel
            sat_level = raw_obj.white_level
            cfa_type = raw_obj.raw_pattern

        # normalize to 0-1
        cfa_data = cfa_data.astype(np.float64)
        cfa_data[cfa_mask == 0] = cfa_data[cfa_mask == 0] - blk_level[0]
        cfa_data[cfa_mask == 1] = cfa_data[cfa_mask == 1] - blk_level[1]
        cfa_data[cfa_mask == 2] = cfa_data[cfa_mask == 2] - blk_level[2]
        cfa_data[cfa_mask == 3] = cfa_data[cfa_mask == 3] - blk_level[3]
        cfa_data = cfa_data / (sat_level - max(blk_level))
        cfa_data = np.clip(cfa_data, 0.0, 1.0)

        # Bayer pattern unification
        cfa_data = unifyBayerPtn(cfa_data, cfa_type)

        # pack to 4-channel raw
        noisy_raw = np.zeros((math.ceil(cfa_data.shape[0] / 2),
                              math.ceil(cfa_data.shape[1] / 2), 4))
        noisy_raw[:, :, 0] = cfa_data[0::2, 0::2]
        noisy_raw[:, :, 1] = cfa_data[0::2, 1::2]
        noisy_raw[:, :, 2] = cfa_data[1::2, 0::2]
        noisy_raw[:, :, 3] = cfa_data[1::2, 1::2]

        # crop to square (even number)
        hei, wid, _ = noisy_raw.shape
        hei = hei // 32 * 32  # to avoid odd number pooling
        wid = wid // 32 * 32  # to avoid odd number pooling
        offset = int(abs(wid - hei) / 2)
        if hei < wid:
            noisy_raw = noisy_raw[:hei, offset:offset + hei, :]
        elif hei > wid:
            noisy_raw = noisy_raw[offset:offset + wid, :wid, :]

        # compute color transform matrix
        metadata = extMetadata(file_path)
        wp_xyz, interp_w = cam2xyzWP(metadata)
        cam2xyz = cam2xyzD50(metadata, wp_xyz, interp_w)

        # convert to tensor
        noisy_raw = t.tensor(noisy_raw.astype(np.float32)).permute(2, 0, 1)
        cam2xyz = t.tensor(cam2xyz.astype(np.float32))

        return noisy_raw, cam2xyz, file

    def __len__(self):

        return len(self.file_list)

## Test set

In [ ]:
class TestSet(data.Dataset):
    def __init__(self, data_root, folder):
        self.data_path = os.path.join(data_root, folder, 'dng')

        # load noise models
        model_path = os.path.join(data_root, folder, 'noiseModel')
        with open(os.path.join(model_path, 'model_params.pkl'), 'rb') as pkl:
            self.noise_params = pickle.load(pkl)

        # find all training samples
        self.file_list = [
            file for file in os.listdir(os.path.join(self.data_path))
            if '.dng' in file or '.DNG' in file
        ]
        self.file_list.sort()

    def __getitem__(self, index):
        # load a new sample
        file = self.file_list[index]
        file_path = os.path.join(self.data_path, file)
        with rp.imread(file_path) as raw_obj:
            cfa_data = raw_obj.raw_image_visible.copy()
            cfa_mask = raw_obj.raw_colors_visible
            blk_level = raw_obj.black_level_per_channel
            sat_level = raw_obj.white_level
            cfa_type = raw_obj.raw_pattern

        # normalize to 0-1
        cfa_data = cfa_data.astype(np.float64)
        cfa_data[cfa_mask == 0] = cfa_data[cfa_mask == 0] - blk_level[0]
        cfa_data[cfa_mask == 1] = cfa_data[cfa_mask == 1] - blk_level[1]
        cfa_data[cfa_mask == 2] = cfa_data[cfa_mask == 2] - blk_level[2]
        cfa_data[cfa_mask == 3] = cfa_data[cfa_mask == 3] - blk_level[3]
        cfa_data = cfa_data / (sat_level - max(blk_level))
        cfa_data = np.clip(cfa_data, 0.0, 1.0)

        # Bayer pattern unification
        cfa_data = unifyBayerPtn(cfa_data, cfa_type)

        # pack to 4-channel raw
        noisy_raw = np.zeros((math.ceil(cfa_data.shape[0] / 2),
                              math.ceil(cfa_data.shape[1] / 2), 4))
        noisy_raw[:, :, 0] = cfa_data[0::2, 0::2]
        noisy_raw[:, :, 1] = cfa_data[0::2, 1::2]
        noisy_raw[:, :, 2] = cfa_data[1::2, 0::2]
        noisy_raw[:, :, 3] = cfa_data[1::2, 1::2]

        # crop to square (even number)
        hei, wid, _ = noisy_raw.shape
        hei = hei // 32 * 32  # to avoid odd number pooling
        wid = wid // 32 * 32  # to avoid odd number pooling
        offset = int(abs(wid - hei) / 2)
        if hei < wid:
            noisy_raw = noisy_raw[:hei, offset:offset + hei, :]
        elif hei > wid:
            noisy_raw = noisy_raw[offset:offset + wid, :wid, :]

        # compute ISO, noise model, and color matrix
        metadata = extMetadata(file_path)
        cam_iso = metadata['iso']
        noise_model = self.noise_params[cam_iso]
        wp_xyz, interp_w = cam2xyzWP(metadata)
        cam2xyz = cam2xyzD50(metadata, wp_xyz, interp_w)

        # compute noise map
        noise_map = noise_model['k'] * noisy_raw + noise_model[
            'sig_r']**2 + noise_model[
                'sig_tl']**2 + noise_model['hf_step']**2 / 3

        # convert to tensor
        noisy_raw = t.tensor(noisy_raw.astype(np.float32)).permute(2, 0, 1)
        noise_map = t.tensor(noise_map.astype(np.float32)).permute(2, 0, 1)
        cam2xyz = t.tensor(cam2xyz.astype(np.float32))

        return noisy_raw, noise_map, cam2xyz, file

    def __len__(self):

        return len(self.file_list)